## Import and Load API key

In [13]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import operator
from typing import TypedDict, Annotated, Literal, Sequence
import os

print("All imports successful")

All imports successful


In [14]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("API key loaded")

API key loaded


In [15]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"LLM initialized: {llm.model_name}")

LLM initialized: gpt-4o-mini


## Quality Score

In [4]:
from pydantic import BaseModel, Field

# Create class for quality score

class QualityScore(BaseModel):
    clarity: int = Field(ge=1, le=5)
    completeness: int = Field(ge=1, le=5)
    accuracy: int = Field(ge=1, le=5)

    def is_approved(self) -> bool:
        return all(score >= 4 for score in self.model_dump().values())
print("QualityScore model defined")

QualityScore model defined


## Create Tool

In [16]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 5" or "2 * 10"
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search(query: str) -> str:
    """
    Search for information (practical).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }
    
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    
    return "No information found."


@tool
def think(reasoning: str) -> str:
    """Use this tool to explicitly record your step-by-step thinking.
    You MUST call this BEFORE using any other tool or giving final answer."""
    return f"Thought recorded: {reasoning}"

tools = [calculator, search, think]
print("Tools created")

Tools created


## Bind tools to LLM

In [18]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Custom State
class ReActTraceState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    reasoning_trace: Annotated[list[str], operator.add]

MessagesState = ReActTraceState

# System prompt
react_prompt = SystemMessage(content="""You are a helpful assistant with step-by-step assistant.

Use calculator for math and search for information.
Think step-by-step before using tools.

You MUST follow this exact pattern for EVERY response:
1. First call the 'think' tool to record your current reasoning
2. Then (and only then) call other tools if needed or give the final answer

Always be very explicit and detailed in your thinking."""
                             )

# Define nodes
def react_assistant(state: MessagesState) -> dict:
    """Agent node - reasons and decides which tool to use."""
    messages = [react_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Route to tools or end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"



# Build graph
react_builder = StateGraph(MessagesState)
react_builder.add_node("assistant", react_assistant)
react_builder.add_node("tools", ToolNode(tools))

react_builder.add_edge(START, "assistant")
react_builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
react_builder.add_edge("tools", "assistant")  # Loop back for multi-step reasoning

react_agent_manual = react_builder.compile(checkpointer=MemorySaver())

print("ReAct agent (manual) created")

ReAct agent (manual) created


## Let us test function

In [19]:
def test_agent(agent, query: str, agent_name: str = "Agent"):
    sep = "═" * 50
    mid = "─" * 20

    print(f"\n{sep}")
    print(f"QUERY: {query}")
    print(f"{sep}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=query)]},
        config={"configurable": {"thread_id": f"test_{agent_name}"}}
    )

    reasoning_trace = []

    print("Explicit thought:", end=" ")

    # First pass: extract first think call (preview)
    for msg in result["messages"]:
        if isinstance(msg, ToolMessage) and msg.name == "think":
            print(msg.content.replace("Thought recorded:", "").strip())
            break

    print("\nFINAL RESULT:\n")

    for msg in result["messages"]:

        # User
        if isinstance(msg, HumanMessage):
            print(f"User: {msg.content}")
            print(mid)

        # Assistant
        elif isinstance(msg, AIMessage):
            print("Assistant:")
            if msg.tool_calls:
                print(f"   Tool calls: {[tc['name'] for tc in msg.tool_calls]}")
            elif msg.content.strip():
                print(f"   {msg.content.strip()}")
            print(mid)

        # Tool output
        elif isinstance(msg, ToolMessage):
            print(f"{msg.name} → {msg.content}")
            print(mid)

            if msg.name == "think":
                reasoning_trace.append(
                    msg.content.replace("Thought recorded:", "").strip()
                )

    print("\nReasoning trace collected:")
    print("...")

    for r in reasoning_trace:
        print("•", r)

    print(f"\n{sep}")
    print("\n(Output is truncated. View as a scrollable)")

    return result["messages"][-1].content

In [20]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 5 + 15 ?",
    "Manual ReAct_76"
)


══════════════════════════════════════════════════
QUERY: I need to calculate 5 + 15 ?
══════════════════════════════════════════════════

Explicit thought: To calculate 5 + 15, I need to add the two numbers together. The sum of 5 and 15 will give me the final result.

FINAL RESULT:

User: I need to calculate 5 + 15 ?
────────────────────
Assistant:
   Tool calls: ['think']
────────────────────
think → Thought recorded: To calculate 5 + 15, I need to add the two numbers together. The sum of 5 and 15 will give me the final result.
────────────────────
Assistant:
   Tool calls: ['calculator']
────────────────────
calculator → 20
────────────────────
Assistant:
   The result of the calculation 5 + 15 is 20.
────────────────────

Reasoning trace collected:
...
• To calculate 5 + 15, I need to add the two numbers together. The sum of 5 and 15 will give me the final result.

══════════════════════════════════════════════════

(Output is truncated. View as a scrollable)


'The result of the calculation 5 + 15 is 20.'

In [22]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 10 * 10 ?",
    "Manual ReAct_77"
)


══════════════════════════════════════════════════
QUERY: I need to calculate 10 * 10 ?
══════════════════════════════════════════════════

Explicit thought: To calculate 12 * 9, I will use a calculator to find the product of these two numbers.

FINAL RESULT:

User: I need to calculate 12 * 9 ?
────────────────────
Assistant:
   Tool calls: ['think']
────────────────────
think → Thought recorded: To calculate 12 * 9, I will use a calculator to find the product of these two numbers.
────────────────────
Assistant:
   Tool calls: ['calculator']
────────────────────
calculator → 108
────────────────────
Assistant:
   The result of 12 * 9 is 108.
────────────────────
User: I need to calculate 10 * 10 ?
────────────────────
Assistant:
   Tool calls: ['think']
────────────────────
think → Thought recorded: To calculate 10 * 10, I will use a calculator to find the product of these two numbers.
────────────────────
Assistant:
   Tool calls: ['calculator']
────────────────────
calculator → 100

'The result of 10 * 10 is 100.'

In [23]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 70 / 0 ?",
    "Manual ReAct_78"
)


══════════════════════════════════════════════════
QUERY: I need to calculate 70 / 0 ?
══════════════════════════════════════════════════

Explicit thought: Dividing by zero is undefined in mathematics. Therefore, calculating 70 / 0 will not yield a valid result.

FINAL RESULT:

User: I need to calculate 70 / 0 ?
────────────────────
Assistant:
   Tool calls: ['think']
────────────────────
think → Thought recorded: Dividing by zero is undefined in mathematics. Therefore, calculating 70 / 0 will not yield a valid result.
────────────────────
Assistant:
   The calculation of \( 70 / 0 \) is undefined in mathematics. Dividing by zero does not produce a valid result.
────────────────────

Reasoning trace collected:
...
• Dividing by zero is undefined in mathematics. Therefore, calculating 70 / 0 will not yield a valid result.

══════════════════════════════════════════════════

(Output is truncated. View as a scrollable)


'The calculation of \\( 70 / 0 \\) is undefined in mathematics. Dividing by zero does not produce a valid result.'